### Installation des dépendances

In [36]:
%pip install numpy
%pip install opencv-python
import numpy as np
import cv2 as cv
import math
import os
import time

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: C:\Users\loicn\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: C:\Users\loicn\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [10]:
# Define Constants
maxFrameIndexationTime = 3 # in seconds
maxSearchTime = 0.1 # in seconds

In [35]:
#Define all global variables
imageList = []
videoList = []
frameIndexationTime = [] # Time must be in seconds
imageSearchTime = [] # Time must be in seconds
videoDurationList = [] # Time must be in seconds
imageCompressionRateList = []
videoCompressionRateList = []

### Lecture des fichiers

In [41]:
imagePath = "/data/jpeg/"
videoPath = "/data/mp4/"
currDirectory = os.getcwd()
imageList = os.listdir(currDirectory + imagePath)
videoList = os.listdir(currDirectory + videoPath)
# for file in imageList:
#     filePath = currDirectory + dataPath + file
#     img = cv.imread(filePath, cv.IMREAD_COLOR)

In [63]:
def calculateTotalFrame():
    """Calculate the total number of frames in all the videos in the videoList."""
    TotalFrame = 0
    for video in videoList:
        path = currDirectory + videoPath + video
        cap = cv.VideoCapture(path)
        TotalFrame += cap.get(cv.CAP_PROP_FRAME_COUNT)

    print (TotalFrame)

calculateTotalFrame()

37895.0


In [ ]:
def createIndexTable():
    

In [ ]:
def indexImage(image):
    

In [ ]:
def createDescriptor():
    

In [ ]:
def videoCompressionRate(indexedVideo, video):

In [ ]:
def imageCompressionRate(indexedImage, image):

In [ ]:
def writeSearchResults():
    

In [ ]:
def searchImage():
    

In [27]:
def verifyFrameIndexationTime():
    """Function to verify if the indexation time of each frame is less than the max time allowed for indexation per frame."""
    videoToReindex = []
    totalIndexationTime = 0
    for i in range(0, len(frameIndexationTime)):
        totalIndexationTime += frameIndexationTime[i]
        if(frameIndexationTime[i] > maxFrameIndexationTime):
            videoToReindex.append(i)
    if len(videoToReindex) != 0:
        print("Les vidéos suivantes doivent être réindexées car elles prennent plus de temps que le temps maximal d'indexation par frame: " + str(maxFrameIndexationTime) + " secondes")
        for i in range(0, len(videoToReindex)):
            print(videoList[videoToReindex[i]])
    print("Toutes les vidéos ont été correctement indexées\nLe temps d'indexation total est: " + str(totalIndexationTime/60.0) + " minutes")


In [50]:
def verifyImageSearchTime():
    """Function to verify if the search time for each image is less than the max time allowed for search per image."""
    imageToResearch = []
    totalSearchTime = 0
    for i in range(0, len(imageSearchTime)):
        totalSearchTime += imageSearchTime[i]
        if(imageSearchTime[i] > maxSearchTime):
            imageToResearch.append(i)
    if len(imageToResearch) != 0:
        print("La recherche doit être révisée pour les images suivantes car elles prennent plus de temps que le temps maximal de recherche par image: " + str(maxSearchTime) + " secondes")
        for i in range(0, len(imageToResearch)):
            print(imageList[imageToResearch[i]])
    print("Le temps de recherche total est: " + str(totalSearchTime) + " secondes")

In [62]:
# Cell for test purposes
calculateTotalFrame()

37895.0
